In [25]:
import cv2 as cv
import numpy as np

from cvzone.FaceDetectionModule import FaceDetector
from cvzone.FaceMeshModule import FaceMeshDetector


In [26]:
leftEye = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]


In [27]:

detector = FaceDetector()
meshDetector = FaceMeshDetector()

cap = cv.VideoCapture('Human Face Stock Footage- Royalty-Free Video Clips.mp4')


# face_img = cv.imread('./19407.JPG')
# face_img = cv.resize(face_img, (300, 400))
# img_copy = face_img.copy()



In [28]:

while cap.isOpened() :
    ret , frame = cap.read()
    ret , frame2 = cap.read()

    if ret == True:
        face_img, bbox = detector.findFaces(frame)
        face_img, faces = meshDetector.findFaceMesh(frame)

        if bbox :
            center = bbox[0]['center']
            if faces:
                leftEyePoints = np.array([[faces[0][p][0], faces[0][p][1]] for p in leftEye])
                # cv.fillPoly(img_copy, pts=[leftEyePoints], color=255)
                (ex,ey,ew,eh) = cv.boundingRect(leftEyePoints)
                # cv.rectangle(img_copy, (ex,ey),(ex+ew, ey+eh), (255,255,255))

                eye_roi = frame2[ey:ey+eh, ex:ex+ew]

                eye_roi_gray = cv.cvtColor(eye_roi, cv.COLOR_BGR2GRAY)
                _, iris = cv.threshold(eye_roi_gray , 40 , 255 , cv.THRESH_BINARY_INV)

                contours, _ = cv.findContours(iris, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
                contours = sorted(contours, key=lambda x: cv.contourArea(x), reverse=True)

                if contours:
                    (ix,iy,iw,ih) = cv.boundingRect(contours[0])
                    ix_center, iy_center = ix+int(iw/2) + ex, iy+int(ih/2) + ey
                    cv.circle(frame2, (ix_center, iy_center), 3, (0,0,255), -1)

                    ix_center_e, iy_center_e = ix+int(iw/2), iy+int(ih/2)

                    
                    offset = 10
                    if ix_center_e > int(iw/2)+offset :
                        text = 'right'
                    elif ix_center_e < int(ih/2)-offset :
                        text = 'left'
                    else:
                        text = 'center'

                    cv.putText(frame2, text, (100, 100), cv.FONT_HERSHEY_PLAIN, 3, (0, 80, 0), 2)


        cv.imshow('frame2', frame2)
        if cv.waitKey(25) & 0xFF == ord('q'):
            break
    
    else:
        break

cap.release()
cv.destroyAllWindows()
           
        